## Text Representation Methods based on word embedding models
word2vec - Google

glove - Stanford

fasttext - Facebook

In [3]:
import gensim.downloader as api

In [4]:
import pandas as pd

In [5]:
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS

In [6]:
import spacy
nlp = spacy.load('en_core_web_sm')

In [7]:
import numpy as np

In [8]:
api.info()

{'corpora': {'semeval-2016-2017-task3-subtaskBC': {'num_records': -1,
   'record_format': 'dict',
   'file_size': 6344358,
   'reader_code': 'https://github.com/RaRe-Technologies/gensim-data/releases/download/semeval-2016-2017-task3-subtaskB-eng/__init__.py',
   'license': 'All files released for the task are free for general research use',
   'fields': {'2016-train': ['...'],
    '2016-dev': ['...'],
    '2017-test': ['...'],
    '2016-test': ['...']},
   'description': 'SemEval 2016 / 2017 Task 3 Subtask B and C datasets contain train+development (317 original questions, 3,169 related questions, and 31,690 comments), and test datasets in English. The description of the tasks and the collected data is given in sections 3 and 4.1 of the task paper http://alt.qcri.org/semeval2016/task3/data/uploads/semeval2016-task3-report.pdf linked in section “Papers” of https://github.com/RaRe-Technologies/gensim-data/issues/18.',
   'checksum': '701ea67acd82e75f95e1d8e62fb0ad29',
   'file_name': 'se

## Data Acquisition

In [9]:
# Pretrained word to vec model
pretrained_w2v_model = api.load('word2vec-google-news-300')

In [10]:
amazon_df = pd.read_csv("amazon_cells_labelled.txt", sep='\t', header=None)
yelp_df = pd.read_csv("yelp_labelled.txt", sep='\t', header=None)
imdb_df = pd.read_csv("imdb_labelled.txt", sep='\t', header=None)

In [11]:
# Combine all datasets into one big dataset
master_df = pd.concat([amazon_df, yelp_df, imdb_df], )

## Text Cleaning/Processing

In [12]:
def clean_preprocess(text):
    # Lowercase
    outtext = text.lower()
    # Remove punctuation and digits
    doc = nlp(outtext)
    outtext = [token.text for token in doc if (not token.is_punct) and (not token.like_num)]
    # remove stop words
    processed_text = [word for word in outtext if word not in ENGLISH_STOP_WORDS]
    #processed_text = ' '.join(outtext) # join list of words into sentences
    return processed_text

In [13]:
master_df[0] = master_df[0].apply(lambda row: clean_preprocess(row))

## Text Representation or Feature Engineering (using word2vec pretrained model)

In [14]:
def convert_word_to_vector(text):
    total_vector = []
    for row in text:# Iterating each sentence in the text column
        count = 0
        row_total_vector = np.zeros(300) # initializing with same dimensions as vector outputs
        # Process each word in each sentence
        for word in row:
            if word in pretrained_w2v_model:
                row_total_vector += pretrained_w2v_model[word]
                count += 1
        if count != 0:
            row_avg_vector = row_total_vector/count
            total_vector.append(row_avg_vector)
        else:
            total_vector.append(np.zeros(300))
    return total_vector


In [15]:
X_vector = convert_word_to_vector(master_df[0])

## Training the Model

In [16]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

In [20]:
X_train, X_test, Y_train, Y_test = train_test_split(X_vector, master_df[1], random_state=1)

In [21]:
model = LogisticRegression() # define the algorithm
model.fit(X_train, Y_train) # train the model

,"penalty penalty: {'l1', 'l2', 'elasticnet', None}, default='l2'Specify the norm of the penalty:- `None`: no penalty is added;- `'l2'`: add a L2 penalty term and it is the default choice;- `'l1'`: add a L1 penalty term;- `'elasticnet'`: both L1 and L2 penalty terms are added... warning:: Some penalties may not work with some solvers. See the parameter `solver` below, to know the compatibility between the penalty and solver... versionadded:: 0.19 l1 penalty with SAGA solver (allowing 'multinomial' + L1).. deprecated:: 1.8 `penalty` was deprecated in version 1.8 and will be removed in 1.10. Use `l1_ratio` instead. `l1_ratio=0` for `penalty='l2'`, `l1_ratio=1` for `penalty='l1'` and `l1_ratio` set to any float between 0 and 1 for `'penalty='elasticnet'`.",'deprecated'
,"C C: float, default=1.0Inverse of regularization strength; must be a positive float.Like in support vector machines, smaller values specify strongerregularization. `C=np.inf` results in unpenalized logistic regression.For a visual example on the effect of tuning the `C` parameterwith an L1 penalty, see::ref:`sphx_glr_auto_examples_linear_model_plot_logistic_path.py`.",1.0
,"l1_ratio l1_ratio: float, default=0.0The Elastic-Net mixing parameter, with `0 <= l1_ratio <= 1`. Setting`l1_ratio=1` gives a pure L1-penalty, setting `l1_ratio=0` a pure L2-penalty.Any value between 0 and 1 gives an Elastic-Net penalty of the form`l1_ratio * L1 + (1 - l1_ratio) * L2`... warning:: Certain values of `l1_ratio`, i.e. some penalties, may not work with some solvers. See the parameter `solver` below, to know the compatibility between the penalty and solver... versionchanged:: 1.8 Default value changed from None to 0.0... deprecated:: 1.8 `None` is deprecated and will be removed in version 1.10. Always use `l1_ratio` to specify the penalty type.",0.0
,"dual dual: bool, default=FalseDual (constrained) or primal (regularized, see also:ref:`this equation `) formulation. Dual formulationis only implemented for l2 penalty with liblinear solver. Prefer `dual=False`when n_samples > n_features.",False
,"tol tol: float, default=1e-4Tolerance for stopping criteria.",0.0001
,"fit_intercept fit_intercept: bool, default=TrueSpecifies if a constant (a.k.a. bias or intercept) should beadded to the decision function.",True
,"intercept_scaling intercept_scaling: float, default=1Useful only when the solver `liblinear` is usedand `self.fit_intercept` is set to `True`. In this case, `x` becomes`[x, self.intercept_scaling]`,i.e. a ""synthetic"" feature with constant value equal to`intercept_scaling` is appended to the instance vector.The intercept becomes``intercept_scaling * synthetic_feature_weight``... note:: The synthetic feature weight is subject to L1 or L2 regularization as all other features. To lessen the effect of regularization on synthetic feature weight (and therefore on the intercept) `intercept_scaling` has to be increased.",1
,"class_weight class_weight: dict or 'balanced', default=NoneWeights associated with classes in the form ``{class_label: weight}``.If not given, all classes are supposed to have weight one.The ""balanced"" mode uses the values of y to automatically adjustweights inversely proportional to class frequencies in the input dataas ``n_samples / (n_classes * np.bincount(y))``.Note that these weights will be multiplied with sample_weight (passedthrough the fit method) if sample_weight is specified... versionadded:: 0.17 *class_weight='balanced'*",None
,"random_state random_state: int, RandomState instance, default=NoneUsed when ``solver`` == 'sag', 'saga' or 'liblinear' to shuffle thedata. See :term:`Glossary ` for details.",None
,"solver solver: {'lbfgs', 'liblinear', 'newton-cg', 'newton-cholesky', 'sag', 'saga'}, default='lbfgs'Algorithm to use in the optimization problem. Default is 'lbfgs'.To choose a solver, you might want to consider the following aspects:- 'lbfgs' is a good default solver because it works reasonably well for a wide class of problems.- For :term:`mul

In [22]:
# test model w test data
y_pred = model.predict(X_test)

## Evaluating the Model

In [23]:
from sklearn.metrics import accuracy_score

accuracy_score(y_pred, Y_test)

0.8296943231441049

# 6. applying, using model

In [34]:
review = "I love this movie"
processed = clean_preprocess(review)
sample_vector = convert_word_to_vector([processed]) # make two dimensional, as it is expecting to get that shape list

In [35]:
sentiment_score = model.predict(sample_vector)
print(sentiment_score)

[1]
